# RAG

## API Keys

In [4]:
from config import METISAI_API_KEY

## Collect Text Files

In [24]:
from pathlib import Path

NUMBER_OF_LASTEST_NEWS = 1000
index_path = "../index/varzesh3.com/news/"

def collect_text_files(root_dir, pattern= "*.md"):
    return list(Path(root_dir).rglob(pattern))

texts_paths = collect_text_files(index_path)

texts_paths.sort()
texts_paths = texts_paths[-NUMBER_OF_LASTEST_NEWS : ]
print('Number of Latest Docs:' , len(texts_paths))

Number of Latest Docs: 1000


In [25]:
from chunker import Chunker

chunker = Chunker()
docs = []

for markdown_path in texts_paths:

    with open(markdown_path, "r", encoding='utf-8') as f:
        markdown = f.read()

    splits = chunker.split_markdown(markdown)
    large_splits = chunker.remove_small_chunks(splits)
    
    docs.extend(large_splits)

docs[-5:]

[Document(metadata={'Header 1': 'جونینیو بعد از استقلال همچنان گل میزند!'}, page_content='با این تساوی، تیم المحرق ۱۱ امتیازی شد و با وجود از دست دادن دو امتیاز خانگی، همچنان صدر جدول لیگ بحرین را حفظ کرد. این تیم قرار است در تاریخ سوم دیماه در آخرین دیدار مرحله گروهی لیگ قهرمانان آسیا میزبان استقلال ایران باشد؛ مسابقهای که اهمیت فراوانی برای هر دو تیم دارد.\nاستقلال برای صعود به مرحله بعدی باید تمامی دیدارهای باقیمانده خود را با پیروزی پشت سر بگذارد و نبرد مقابل المحرق در منامه، بدون تردید یکی از سختترین بازیهای این تیم در مسیر آسیاییاش خواهد بود.'),
 Document(metadata={'Header 1': 'عقیلی: مطمئنا سپاهان مدعی قهرمانی است'}, page_content='# عقیلی: مطمئنا سپاهان مدعی قهرمانی است\nمربی سپاهان از نمایش تیمش مقابل استقلال خوزستان ابراز رضایت کرد و تأکید داشت تیمش در مسیر قهرمانی قرار دارد.\nبه گزارش "ورزش سه"،'),
 Document(metadata={'Header 1': 'تساوی بدون گل در حضور شش ستاره ایرانی!'}, page_content='# تساوی بدون گل در حضور شش ستاره ایرانی!\nدر روزی که 6 بازیکن ایرانی برای دو تیم کلباء و شب

In [26]:
len(docs)

1877

## Vector Store

In [7]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from pydantic import SecretStr

# Set UP the link
vector_store = Chroma(
    collection_name="test",
    embedding_function=OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=SecretStr(METISAI_API_KEY),
        base_url="https://api.metisai.ir/openai/v1",
    ),
)

for i in range(int(len(docs) / 250 + 1)):
    vector_store.add_documents(docs[i*250: (i+1)*250], ids=[str(j + i * 250) for j in range(len(docs))])

/var/folders/56/2bx9230j77nckch23fg26xh00000gn/T/ipykernel_6564/1392131427.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [8]:
vector_store._collection.count()

1416

## LLM

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key=SecretStr(METISAI_API_KEY),
    base_url="https://api.metisai.ir/openai/v1",
    temperature=0.1,
    verbose=True,
)

In [10]:
a = llm.invoke("ارلینگ هالند چند گل ملی زده است؟")
print(a.content)

تا تاریخ اکتبر 2023، ارلینگ هالند حدود 26 گل ملی برای تیم ملی نروژ زده است. لطفاً توجه داشته باشید که این آمار ممکن است تغییر کرده باشد، بنابراین برای اطلاعات به‌روزتر، به منابع رسمی مراجعه کنید.


## Prompt

In [ ]:
from langchain import PromptTemplate

template = """
لطفا بر اساس مطالب داده شده به سوال پاسخ بده

مطالب: {context}
سوال: {question}
جواب:
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

## Chain

In [12]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": vector_store.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [13]:
result = rag_chain.invoke("ارلینگ هالند چند گل ملی زده است؟")
print(result)

ارلینگ هالند در حال حاضر ۵۰ گل ملی زده است.


## Accuracy

In [24]:
def rag_answer(question):

    return rag_chain.invoke(question)

def llm_answer(question):
    
    return llm.invoke(question).content

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

development_questions_csv_path = '../questions/IctFootballQuestions 2024-10-13.csv'
development_questions_csv = pd.read_csv(development_questions_csv_path)
development_questions_csv

,Category,Question,Answer,GPT 5 with Search,GPT 4.1 nano,ICT-LLM
0,Board,تیم اول جدول لیگ برتر چیه؟,تراکتور,Right,Wrong,Wrong
1,Board,تیم ۱۵ ام لیگ برتر کیه؟,مس رفسنجان,Wrong,Wrong,Wrong
2,Board,استقلال تیم چندم جدول هست؟,تیم هشتم,Wrong,Wrong,Wrong
3,Board,برترین گلزن لیگ تا به اینجا کیست؟,کسری طاهری و علی علیپور با ۴ گل,Right,Wrong,Wrong
4,Match,نتیجه بازی آخر تیم ‌ذوب آهن چه شد؟,تساوی صفر صفر برابر سپاهان,Wrong,Wrong,Wrong
5,Match,نتیجه بازی آخر تیم ملوان چه شد؟,برد ۱ بر هیچ برابرخیبر خرم آباد,Wrong,Wrong,Wrong
6,Match,آخرین بازی تیم پرسپولیس مقابل چه تیمی بود؟,گل گهر سیرجان,Wrong,Wrong,Wrong
7,Match,آخرین بازی تیم خیبر خرم آباد مقابل چه تیمی بود؟,ملوان,Wrong,Wrong,Wrong
8,Goal,گل های بازی فجرسپاسی و استقلال خوزستان توسط چه...,محمود مطلق زاده برای فجر سپاسی و محمدرضا مهدی ...,Right,Wrong,Wrong
9,Goal,گل های بازی ملوان و خیبر خرم آباد توسط چه کسان...,ابوذر صفر زاده ۱ گل برای ملوان,Right,Wrong,Wrong


In [22]:
development_questions_csv['prediction'] = development_questions_csv['Question'].progress_map(rag_answer)
development_questions_csv

  0%|          | 0/25 [00:00<?, ?it/s]

,Category,Question,Answer,GPT Answer,prediction
0,Board,تیم اول جدول لیگ برتر چیه؟,تراکتور,Right,با توجه به مطالب ارائه شده، اطلاعات دقیقی دربا...
1,Board,تیم ۱۵ ام لیگ برتر کیه؟,مس رفسنجان,Wrong,در مطالب ارائه شده، هیچ اشاره‌ای به تیمی با رت...
2,Board,استقلال تیم چندم جدول هست؟,تیم هشتم,Right,با توجه به مطالب ارائه شده، اطلاعات دقیقی دربا...
3,Board,برترین گلزن لیگ تا به اینجا کیست؟,کسری طاهری و علی علیپور با ۴ گل,Wrong,بر اساس مطالب داده شده، حسین‌زاده به عنوان آقا...
4,Match,نتیجه بازی آخر تیم ‌ذوب آهن چه شد؟,تساوی صفر صفر برابر سپاهان,Right,نتیجه بازی آخر تیم ذوب‌آهن در متن ذکر نشده است.
5,Match,نتیجه بازی آخر تیم ملوان چه شد؟,برد ۱ بر هیچ برابرخیبر خرم آباد,Right,نتیجه بازی آخر تیم ملوان در مطالب ارائه شده ذک...
6,Match,آخرین بازی تیم پرسپولیس مقابل چه تیمی بود؟,گل گهر سیرجان,Right,آخرین بازی تیم پرسپولیس مقابل تیم عقاب بود.
7,Match,آخرین بازی تیم خیبر خرم آباد مقابل چه تیمی بود؟,ملوان,Right,آخرین بازی تیم خیبر خرم آباد مقابل تیم پرسپولی...
8,Goal,گل های بازی فجرسپاسی و استقلال خوزستان توسط چه...,محمود مطلق زاده برای فجر سپاسی و محمدرضا مهدی ...,Right,در مطلب ارائه شده، نام افراد یا بازیکنانی که گ...
9,Goal,گل های بازی ملوان و خیبر خرم آباد توسط چه کسان...,ابوذر صفر زاده ۱ گل برای ملوان,Right,در مطالب ارائه شده، هیچ اطلاعاتی درباره گل‌های...


In [25]:
development_questions_csv['gpt_api'] = development_questions_csv['Question'].progress_map(llm_answer)
development_questions_csv

  0%|          | 0/25 [00:00<?, ?it/s]

,Category,Question,Answer,GPT Answer,prediction,gpt_api
0,Board,تیم اول جدول لیگ برتر چیه؟,تراکتور,Right,با توجه به مطالب ارائه شده، اطلاعات دقیقی دربا...,متأسفانه من نمی‌توانم وضعیت زنده و به‌روز جدول...
1,Board,تیم ۱۵ ام لیگ برتر کیه؟,مس رفسنجان,Wrong,در مطالب ارائه شده، هیچ اشاره‌ای به تیمی با رت...,تا تاریخ آخرین بروزرسانی من در اکتبر ۲۰۲۳، تیم...
2,Board,استقلال تیم چندم جدول هست؟,تیم هشتم,Right,با توجه به مطالب ارائه شده، اطلاعات دقیقی دربا...,متأسفانه، من نمی‌توانم وضعیت فعلی جدول رده‌بند...
3,Board,برترین گلزن لیگ تا به اینجا کیست؟,کسری طاهری و علی علیپور با ۴ گل,Wrong,بر اساس مطالب داده شده، حسین‌زاده به عنوان آقا...,متأسفانه، من تا تاریخ اکتبر ۲۰۲۳ به‌روزرسانی ش...
4,Match,نتیجه بازی آخر تیم ‌ذوب آهن چه شد؟,تساوی صفر صفر برابر سپاهان,Right,نتیجه بازی آخر تیم ذوب‌آهن در متن ذکر نشده است.,متأسفانه من اطلاعات به‌روز در مورد نتایج بازی‌...
5,Match,نتیجه بازی آخر تیم ملوان چه شد؟,برد ۱ بر هیچ برابرخیبر خرم آباد,Right,نتیجه بازی آخر تیم ملوان در مطالب ارائه شده ذک...,متأسفانه من اطلاعات به‌روز در مورد نتایج بازی‌...
6,Match,آخرین بازی تیم پرسپولیس مقابل چه تیمی بود؟,گل گهر سیرجان,Right,آخرین بازی تیم پرسپولیس مقابل تیم عقاب بود.,آخرین بازی تیم پرسپولیس در زمان آخرین به‌روزرس...
7,Match,آخرین بازی تیم خیبر خرم آباد مقابل چه تیمی بود؟,ملوان,Right,آخرین بازی تیم خیبر خرم آباد مقابل تیم پرسپولی...,متأسفانه من اطلاعات به‌روز در مورد نتایج بازی‌...
8,Goal,گل های بازی فجرسپاسی و استقلال خوزستان توسط چه...,محمود مطلق زاده برای فجر سپاسی و محمدرضا مهدی ...,Right,در مطلب ارائه شده، نام افراد یا بازیکنانی که گ...,در بازی بین تیم‌های فجر سپاسی و استقلال خوزستا...
9,Goal,گل های بازی ملوان و خیبر خرم آباد توسط چه کسان...,ابوذر صفر زاده ۱ گل برای ملوان,Right,در مطالب ارائه شده، هیچ اطلاعاتی درباره گل‌های...,متأسفانه من اطلاعات دقیقی درباره گل‌های بازی ب...
